In [3]:
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout , MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
import cv2
import os
import random
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.applications import VGG19
from ipykernel import kernelapp as app

In [4]:
labels = ['dandelion', 'daisy','tulip','sunflower','rose']
img_size = 224
def get_data(data_dir):
    data = [] 
    for label in labels: 
        path = os.path.join(data_dir, label)
        class_num = labels.index(label)
        for img in os.listdir(path):
            try:
                img_arr = cv2.imread(os.path.join(path, img), cv2.IMREAD_COLOR)
                resized_arr = cv2.resize(img_arr, (img_size, img_size))
                data.append([resized_arr, class_num])
            except Exception as e:
                print(e)
    return np.array(data)

In [5]:
flower_data = get_data("../input/flower-dataset/flowers")

In [6]:
l = []
for i in flower_data:
    l.append(labels[i[1]])
sns.set_style()
sns.countplot(l)

In [7]:
fig,ax=plt.subplots(2,2)
fig.set_size_inches(8,8)
for i in range(2):
    for j in range (2):
        l=random.randint(0,len(flower_data))
        ax[i,j].imshow(flower_data[l][0])
        ax[i,j].set_title('Flower: '+labels[flower_data[l][1]])
        
plt.tight_layout()

In [8]:
x = []
y = []

for feature, label in flower_data:
    x.append(feature)
    y.append(label)

In [9]:
x = np.array(x) / 255
x = x.reshape(-1, img_size, img_size, 3)
y = np.array(y)

In [10]:
label_binarizer = LabelBinarizer()
y = label_binarizer.fit_transform(y)

In [11]:
x_train,x_test,y_train,y_test = train_test_split(x , y , test_size = 0.2 , stratify = y , random_state = 0)

In [12]:
del x,y,flower_data

In [13]:
# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

In [14]:
pre_trained_model = VGG19(input_shape=(224,224,3), include_top=False, weights="imagenet")

for layer in pre_trained_model.layers[:19]:
  layer.trainable = False

model = Sequential([
      pre_trained_model,
      MaxPool2D((2,2) , strides = 2),
      Flatten(),
      Dense(5 , activation='softmax')])

model.compile(optimizer = "adam" , loss = 'categorical_crossentropy' , metrics = ['accuracy'])

model.summary()

In [15]:
history = model.fit(x_train,y_train, batch_size = 64 , epochs = 15 , validation_data = (x_test, y_test))

In [16]:
print("Loss of the model is - " , model.evaluate(x_test,y_test)[0])
print("Accuracy of the model is - " , model.evaluate(x_test,y_test)[1]*100 , "%")

In [23]:
fig , ax = plt.subplots(1,2)
fig.set_size_inches(20,10)

ax[0].plot(history.history['accuracy'])
ax[0].plot(history.history['val_accuracy'])
ax[0].set_title('Training & Testing Accuracy')
ax[0].set_ylabel('Accuracy')
ax[0].set_xlabel('Epochs')
ax[0].legend(['Train', 'Test'], loc='upper left')

ax[1].plot(history.history['loss'])
ax[1].plot(history.history['val_loss'])
ax[1].set_title('Training & Testing Loss')
ax[1].set_ylabel('Loss')
ax[1].set_xlabel('Epochs')
ax[1].legend(['Train', 'Test'], loc='upper left')

plt.show()

In [24]:
predictions = model.predict_classes(x_test)
predictions[:5]

In [25]:
y_test_inv = label_binarizer.inverse_transform(y_test)

In [26]:
print(classification_report(y_test_inv, predictions, target_names = labels))

In [27]:
cm = confusion_matrix(y_test_inv,predictions)
cm = pd.DataFrame(cm , index = labels , columns = labels)
plt.figure(figsize = (10,10))
sns.heatmap(cm,cmap= "Blues", linecolor = 'black' , linewidth = 1 , annot = True, fmt='' , xticklabels = labels , yticklabels = labels)

In [34]:
import random

fig,ax=plt.subplots(4,2)
fig.set_size_inches(20,20)

for i in range (4):
    for j in range (2):
        number = random.randint(0, x_test.shape[0])
        ax[i,j].imshow(x_test[number])
        ax[i,j].set_title("Predicted Flower : "+ labels[predictions[number]] +"\n"+"Actual Flower : "+ labels[y_test_inv[number]])
        plt.tight_layout()
